# Merge CIRI data with scraped State Dep Reports
## Merge on year and country name

In [1]:
import pandas as pd

In [4]:
# CIRI data
ciri_df = pd.read_csv('../data/CIRI_DATA_2016.csv')
# State Dept. data
#state_dep = pd.read_csv('../data/Custom_State_Dep_Reports/us_state_dept_reports_1999_2018.csv')
state_dep = pd.read_csv('../data/us_state_dept_hr_reports_un_coded_1999_2018.csv')
#state_dep = state_dep.drop(state_dep.columns[0],axis=1)
# Read in reference.

ref = pd.read_csv('../data/Custom_State_Dep_Reports/state_dept_country_names.csv')

ref = ref.drop(ref.columns[0],axis=1)
# Read in GapMinder
codes = pd.read_csv('../data/GapMinder_Raw_CSVs/UNCTRY_CODES_GapMinderNames.csv')

In [5]:
#!!DO NOT RUN AGAIN!!
#Create a dataframe to quickly be cleaned manually.
#ctry = state_dep.Country.unique()
#ctry = pd.DataFrame({"country":ctry, "simple":ctry})
#ctry.to_csv("../data/Custom_State_Dep_Reports/state_dept_country_names.csv")

In [6]:
# Countries in the GapMinder Data are not exactly the same as in the UN codes.
# This function looks for the most parsimonious name link between ...
# ... the GapMinder and UN data and return the UN codes.
def compare(x, codes):
    ctry = {'size':100,'un':0}
    for i in range(len(codes)):
        if x.lower() in codes.CTRY.iloc[i].lower():
            if len(codes.CTRY.iloc[i]) <= ctry['size']:
                ctry['size'] = len(codes.CTRY.iloc[i])
                ctry['un'] = codes.UNCTRY.iloc[i]
    return ctry

In [7]:
# Take the simplified state deptartment country names...
countries = ref.simple.unique()
un = []
# Loop through and find the GapMinder equivalent and UN number.
for i in countries:
    un.append(compare(i, codes)['un'])
codes = pd.DataFrame({'CTRY':countries,'UNCTRY':un})

In [8]:
# Associate the UN numbers by simple country name in state dept. data.
# This has the side-effect of giving the same UN code to various spellings of countries.
st_dpt_un = pd.merge(ref,codes,
                     left_on='simple',
                     right_on='CTRY',how='left').drop('simple',axis=1)

In [9]:
state_dep.columns

Index(['Unnamed: 0', 'year', 'country', 'url', 'text', 'code'], dtype='object')

In [10]:
# Associate complex country name with the State Dept. to include the text data.
state_dep = pd.merge(state_dep,st_dpt_un,
         left_on='country',
         right_on='country',how='left').drop(['CTRY'], axis=1) #original dropped 'country' as well, leaving it in to check its working

In [11]:
state_dep.columns

Index(['Unnamed: 0', 'year', 'country', 'url', 'text', 'code', 'UNCTRY'], dtype='object')

In [12]:
# Merge CIRI data with State Dept. data.
df_text_to_ciri = pd.merge(state_dep,ciri_df,
                           left_on=['UNCTRY','year'],
                           right_on=['unctry',"year"],how = 'inner') #.drop('Country',axis=1)

In [13]:
# Write to file
df_text_to_ciri.to_csv('../data/Custom_State_Dep_Reports/CIRI_Text_1999_2015.csv')